# BTC Time Series Momentum Strategy

## Academic Foundation

Based on **Moskowitz, Ooi & Pedersen (2012)** "Time Series Momentum", this strategy exploits the persistence of price trends (momentum) in financial markets.

**Core principle**: Assets that have performed well recently (1-12 month lookback) tend to continue their trajectory in the short/medium term.

### Mathematical Framework (MOP 2012)

The canonical TSM signal for asset $i$ at time $t$:

$$h_{i,t} = \text{sign}(r_{i,t-12,t}) \times \frac{\sigma_{\text{target}}}{\sigma_{i,t}}$$

Where:
- $r_{i,t-12,t} = P_{i,t}/P_{i,t-12} - 1$: 12-month return (momentum signal)
- $\sigma_{i,t}$: realized volatility (EWMA or rolling)
- $\sigma_{\text{target}}$: target volatility (e.g., 40% annualized)
- $h_{i,t}$: position size (leverage-adjusted)

**Return attribution**:

$$r_{p,t} = \sum_{i} h_{i,t-1} \times r_{i,t} - \text{TC}_{i,t}$$

Where $\text{TC}_{i,t} = c \times |h_{i,t} - h_{i,t-1}|$ are transaction costs.

## Our Implementation

We extend the canonical TSM with crypto-specific enhancements:

1. **Momentum Signal**: $r_{t-\text{lookback},t}$ with lookback ∈ [180, 252, 365] days
2. **Trend Filter**: SMA filter to avoid bear markets (long only if $P_t > \text{SMA}_{\text{trend}}$)
3. **Carry (Funding)**: monetize perpetual funding rates (crypto-specific alpha)
4. **Vol-Targeting**: dynamic leverage to target 35% annualized vol
5. **Position Smoothing**: EWMA halflife 2-4 to reduce turnover
6. **Transaction Costs**: realistic 2bps per trade

**Objective**: beat Buy & Hold at equal risk (same annualized volatility).


In [32]:
import subprocess
import sys
import os
from datetime import datetime,timedelta,timezone
from typing import Optional,List,Tuple,Dict
from dataclasses import dataclass
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ccxt
pd.options.display.float_format='{:.6f}'.format


## Data Loading

Fetch daily BTC/USDT prices since 2017 via CCXT (Binance) and perpetual funding rates.

**Funding rates** are fetched from Binance perpetual futures (paid every 8h). They represent the cost of holding a leveraged position and provide a carry signal.


In [33]:
def fetch_btc_ohlcv(symbol='BTC/USDT',timeframe='1d',since='2017-01-01'):
    ex=ccxt.binance({'enableRateLimit':True})
    since_ms=int(pd.Timestamp(since,tz='UTC').timestamp()*1000)
    all_ohlcv=[]
    while True:
        ohlcv=ex.fetch_ohlcv(symbol,timeframe,since=since_ms,limit=1000)
        if not ohlcv:break
        all_ohlcv.extend(ohlcv)
        since_ms=ohlcv[-1][0]+1
        if len(ohlcv)<1000:break
    df=pd.DataFrame(all_ohlcv,columns=['timestamp','open','high','low','close','volume'])
    df['timestamp']=pd.to_datetime(df['timestamp'],unit='ms',utc=True)
    df.set_index('timestamp',inplace=True)
    return df

def fetch_binance_funding_rates(symbol='BTC/USDT:USDT',limit=1000):
    ex=ccxt.binance({'enableRateLimit':True,'options':{'defaultType':'future'}})
    rates=[]
    since=int((datetime.now(tz=timezone.utc)-timedelta(days=365*3)).timestamp()*1000)
    while True:
        try:
            batch=ex.fetch_funding_rate_history(symbol,since=since,limit=limit)
            if not batch:break
            rates.extend(batch)
            since=batch[-1]['timestamp']+1
            if len(batch)<limit:break
        except:break
    if not rates:return pd.Series(dtype=float)
    df=pd.DataFrame(rates)
    df['datetime']=pd.to_datetime(df['timestamp'],unit='ms',utc=True)
    df.set_index('datetime',inplace=True)
    return df['fundingRate']

df_btc=fetch_btc_ohlcv()
print(f"BTC data: {df_btc.index.min()} -> {df_btc.index.max()} ({len(df_btc)} rows)")


BTC data: 2017-08-17 00:00:00+00:00 -> 2025-11-01 00:00:00+00:00 (2999 rows)


## TSM Strategy: compute_tsm_signals

### Strategy Logic with Mathematical Formulation

**Step 1: Momentum Signal**

$$\text{mom}_t = \frac{P_t}{P_{t-\text{lookback}}} - 1$$

**Step 2: Carry Signal (Crypto-Specific)**

Perpetual futures funding rates provide a carry component:

$$\text{carry}_t = \sum_{s=t-w}^{t} f_s$$

Where $f_s$ is the funding rate at time $s$ (typically paid every 8h on Binance), and $w$ is the carry window (7-30 days).

**Funding Rate Mechanics**:
- **Positive funding** ($f > 0$): longs pay shorts → bullish sentiment → we capture this premium when long
- **Negative funding** ($f < 0$): shorts pay longs → bearish sentiment → we earn funding when long
- **Our strategy**: aggregate funding over `carry_window` days and combine with momentum

**Step 3: Composite Score**

$$\text{score}_t = w_{\text{mom}} \times z(\text{mom}_t) + w_{\text{carry}} \times z(\text{carry}_t)$$

Where $z(x) = (x - \mu_x)/\sigma_x$ is the z-score normalization.

**Step 4: Directional Signal**

$$\text{signal}_t = \begin{cases} 1 & \text{if } \text{score}_t > \theta \text{ and } P_t > \text{SMA}_{\text{trend}}(P) \\ 0 & \text{otherwise} \end{cases}$$

**Step 5: Volatility Targeting**

$$\sigma_{t} = \sqrt{\text{EWMA}_{\lambda}(r_t^2)} \times \sqrt{252}$$

$$\text{leverage}_t = \min\left(\frac{\sigma_{\text{target}}}{\sigma_t}, \text{lev}_{\text{max}}\right)$$

**Step 6: Position with Smoothing**

$$\text{pos}_t^{\text{raw}} = \text{signal}_{t-1} \times \text{leverage}_{t-1}$$

$$\text{pos}_t = \text{EWMA}_{\text{halflife}}(\text{pos}_t^{\text{raw}})$$

**Step 7: Net Return**

$$r_{p,t} = \text{pos}_{t-1} \times r_{t} - c \times |\text{pos}_t - \text{pos}_{t-1}| + \text{pos}_{t-1} \times f_t$$

Where:
- $r_t$: asset return
- $c$: transaction cost (2bps)
- $f_t$: funding rate received/paid

### Key Parameters

- `lookback_days`: momentum window (180-365)
- `trend_sma_days`: SMA for trend filter (150-252)
- `carry_window`: rolling sum window for funding (7-30 days)
- `w_mom`, `w_carry`: momentum vs carry weights (0.7/0.3)
- `score_threshold`: entry threshold (0.0 or 0.25)
- `target_vol`: target annualized volatility (35%)
- `max_leverage`: maximum leverage (3x)
- `position_ewm_halflife`: position smoothing (2-4)
- `transaction_cost_bps`: transaction costs (2bps)


In [34]:
def _z(x:pd.Series)->pd.Series:
    s=x.dropna();v=s.std()
    if v==0 or np.isnan(v):return x*0.0
    return (x-s.mean())/v

def compute_tsm_signals(prices:pd.Series,funding:Optional[pd.Series]=None,rebalance_freq:str='W',lookback_days:int=252,trend_sma_days:int=200,carry_window:int=14,w_mom:float=0.7,w_carry:float=0.3,score_threshold:float=0.0,vol_lambda:float=0.97,target_vol:float=0.35,vol_floor:float=0.15,max_leverage:float=3.0,transaction_cost_bps:float=2.0,position_ewm_halflife:Optional[float]=2.0)->pd.DataFrame:
    prices=prices.dropna().sort_index()
    r=prices.pct_change().dropna()
    ewma=np.sqrt(r.pow(2).ewm(alpha=1-vol_lambda,adjust=False).mean())
    sig_ann=(ewma*np.sqrt(252.0)).clip(lower=vol_floor)
    idx=prices.resample(rebalance_freq).last().index
    df=pd.DataFrame(index=idx)
    df['close']=prices.reindex(idx,method='ffill')
    df['asset_return']=df['close'].pct_change().fillna(0.0)
    mom=prices.pct_change(lookback_days).reindex(idx,method='ffill')
    trend=prices.rolling(trend_sma_days).mean().reindex(idx,method='ffill')
    if funding is not None and not funding.empty:
        carry=funding.rolling(f'{carry_window}D').sum().reindex(idx,method='ffill').fillna(0.0)
    else:
        carry=pd.Series(0.0,index=idx)
    score=w_mom*_z(mom)+w_carry*_z(carry)
    signal=((score>score_threshold)&(df['close']>trend)).astype(float).shift(1).fillna(0.0)
    sigma=sig_ann.resample(rebalance_freq).last().reindex(idx).shift(1).ffill().clip(lower=vol_floor)
    lev=(target_vol/sigma).replace([np.inf,-np.inf],np.nan)
    pos=(signal*lev).clip(-max_leverage,max_leverage).fillna(0.0)
    if position_ewm_halflife:pos=pos.ewm(halflife=position_ewm_halflife,adjust=False).mean()
    df['position']=pos
    df['turnover']=pos.diff().abs().fillna(pos.abs());cost=transaction_cost_bps/10000.0
    df['transaction_cost']=df['turnover']*cost
    fund=pd.Series(0.0,index=idx) if funding is None else funding.resample(rebalance_freq).sum().reindex(idx).fillna(0.0)
    df['funding_pnl']=fund*pos
    df['strategy_return_net']=pos*df['asset_return']-df['transaction_cost']+df['funding_pnl']
    df['strategy_equity']=(1.0+df['strategy_return_net']).cumprod()
    return df

@dataclass
class ParamSet:
    lookback_days:int;trend_sma_days:int;carry_window:int;w_mom:float;w_carry:float;score_threshold:float;rebalance_freq:str='W';vol_lambda:float=0.97;target_vol:float=0.35;vol_floor:float=0.15;max_leverage:float=3.0;transaction_cost_bps:float=2.0;position_ewm_halflife:Optional[float]=2.0
    def to_kwargs(self)->Dict:
        return {'lookback_days':self.lookback_days,'trend_sma_days':self.trend_sma_days,'carry_window':self.carry_window,'w_mom':self.w_mom,'w_carry':self.w_carry,'score_threshold':self.score_threshold,'rebalance_freq':self.rebalance_freq,'vol_lambda':self.vol_lambda,'target_vol':self.target_vol,'vol_floor':self.vol_floor,'max_leverage':self.max_leverage,'transaction_cost_bps':self.transaction_cost_bps,'position_ewm_halflife':self.position_ewm_halflife}

def generate_param_grid()->List[ParamSet]:
    g=[]
    for lb in [180,252,365]:
        for tr in [150,200,252]:
            for cw in [7,14,30]:
                for thr in [0.0,0.25]:
                    g.append(ParamSet(lb,tr,cw,0.7,0.3,thr))
    return g

def compute_sharpe(returns:pd.Series,rebalance_freq:str)->float:
    s=returns.dropna()
    if s.empty:return float('nan')
    ppy=52 if rebalance_freq.upper().startswith('W') else 12
    ann=((1.0+s).prod()**(ppy/len(s)))-1.0
    vol=s.std()*np.sqrt(ppy)
    return float(ann/vol) if vol>0 else float('nan')


## Walk-Forward Analysis (WFA)

### Why WFA?

Classical backtesting (optimize on the entire period) leads to **overfitting**: parameters are over-tuned to historical data and don't generalize.

**Walk-Forward Analysis** simulates real trading:
1. **Train**: optimize parameters on 3 years of past data
2. **Test (OOS)**: apply best parameters on the next 6 months (out-of-sample)
3. **Roll forward**: shift the window and repeat

This ensures parameters are chosen **before** each test period, as in real trading.

### Mathematical Formulation

For each rolling window $w$:

$$\theta_w^* = \arg\max_{\theta \in \Theta} \text{Sharpe}(r_{p,t}(\theta), t \in [t_{\text{train}}^w, t_{\text{train}}^w + T_{\text{train}}])$$

Then evaluate OOS:

$$r_{p,t}^{\text{OOS}} = r_{p,t}(\theta_w^*), \quad t \in [t_{\text{test}}^w, t_{\text{test}}^w + T_{\text{test}}]$$

Aggregate all OOS returns:

$$\text{Sharpe}_{\text{OOS}} = \text{Sharpe}\left(\bigcup_{w} r_{p,t}^{\text{OOS},w}\right)$$

### Implementation

- `generate_param_grid()`: generates 54 parameter combinations (3 lookbacks × 3 trends × 3 carry windows × 2 thresholds)
- `grid_search_on_train()`: tests all parameters on train period, returns best (max Sharpe)
- `walk_forward()`: executes full WFA, returns aggregated OOS returns


In [35]:
def _compute_signals_with_context(prices:pd.Series,test_start:pd.Timestamp,test_end:pd.Timestamp,params:ParamSet,funding:Optional[pd.Series]=None)->pd.DataFrame:
    ctx_start=prices.index.min();pr=prices.loc[ctx_start:test_end]
    fd=None if funding is None else funding.loc[ctx_start:test_end]
    sig=compute_tsm_signals(pr,fd,**params.to_kwargs())
    return sig.loc[test_start:test_end]

def grid_search_on_train(prices:pd.Series,train_start:pd.Timestamp,train_end:pd.Timestamp,param_grid:List[ParamSet],funding:Optional[pd.Series]=None)->Tuple[Optional[ParamSet],pd.DataFrame]:
    rec=[]
    for p in param_grid:
        sig=compute_tsm_signals(prices.loc[train_start:train_end],funding,**p.to_kwargs())
        sh=compute_sharpe(sig['strategy_return_net'],p.rebalance_freq) if not sig.empty else np.nan
        rec.append({'params':p,'sharpe_train':sh})
    df=pd.DataFrame(rec).sort_values('sharpe_train',ascending=False)
    best=None if df.empty else df.iloc[0]['params']
    return best,df

def walk_forward(prices:pd.Series,train_years:int=3,test_months:int=6,param_grid:Optional[List[ParamSet]]=None,funding:Optional[pd.Series]=None):
    if param_grid is None:param_grid=generate_param_grid()
    start=prices.index.min();end=prices.index.max()
    oos=[];rows=[];cursor=start+pd.DateOffset(years=train_years)
    while cursor<end:
        tr_start=cursor-pd.DateOffset(years=train_years);tr_end=cursor-pd.DateOffset(days=1)
        te_start=cursor;te_end=min(te_start+pd.DateOffset(months=test_months)-pd.DateOffset(days=1),end)
        best,_tbl=grid_search_on_train(prices,tr_start,tr_end,param_grid,funding)
        if best is None:break
        sig=_compute_signals_with_context(prices,te_start,te_end,best,funding)
        if sig.empty:
            cursor=te_end+pd.DateOffset(days=1);continue
        oos.append(sig['strategy_return_net'])
        rows.append({'train_start':tr_start,'train_end':tr_end,'test_start':te_start,'test_end':te_end})
        cursor=te_end+pd.DateOffset(days=1)
    oos=pd.concat(oos).sort_index() if oos else pd.Series(dtype=float)
    return pd.DataFrame(rows),oos


## WFA Execution and Buy & Hold Comparison

### Comparison Methodology

We compare the TSM strategy to two benchmarks:

1. **Buy & Hold (BH)**: buy and hold BTC over the entire OOS period
2. **Buy & Hold Equal Risk (BH EQRisk)**: BH scaled to have the same annualized volatility as TSM

Equal-risk comparison is crucial: a higher Sharpe at the same vol means better risk-adjusted return.

### Key Metrics

**Sharpe Ratio**:

$$\text{Sharpe} = \frac{\mu_p - r_f}{\sigma_p} \approx \frac{\mu_p}{\sigma_p}$$

Where $\mu_p$ is annualized return and $\sigma_p$ is annualized volatility (assuming $r_f \approx 0$ for crypto).

**Annualized Return** (geometric):

$$\mu_p = \left(\prod_{t=1}^{T} (1 + r_{p,t})\right)^{N/T} - 1$$

Where $N$ is periods per year (52 for weekly).

**Annualized Volatility**:

$$\sigma_p = \text{std}(r_{p,t}) \times \sqrt{N}$$

**Max Drawdown**:

$$\text{MDD} = \max_{t} \left(\frac{\text{Peak}_t - \text{Equity}_t}{\text{Peak}_t}\right)$$

The Plotly chart displays normalized equity curves (starting at 1) for TSM, BH, and BH EQRisk.


In [36]:
rb='W';ppy=52
fund=None
try:
    _fund=fetch_binance_funding_rates('BTC/USDT:USDT');fund=_fund
except:pass
segments,oos_returns=walk_forward(df_btc['close'],train_years=3,test_months=6,param_grid=generate_param_grid(),funding=fund)
bh_close=df_btc['close'].resample(rb).last().reindex(oos_returns.index,method='ffill')
bh=bh_close.pct_change().fillna(0.0)
vol_tsm=oos_returns.std()*np.sqrt(ppy);vol_bh=bh.std()*np.sqrt(ppy)
scale=vol_tsm/vol_bh if vol_bh>0 else 1.0
bh_eq=bh*scale
sh_tsm=compute_sharpe(oos_returns,rb);sh_bh=compute_sharpe(bh,rb);sh_bh_eq=compute_sharpe(bh_eq,rb)
fig=go.Figure()
fig.add_trace(go.Scatter(x=oos_returns.index,y=(1+oos_returns).cumprod(),name='TSM'))
fig.add_trace(go.Scatter(x=bh.index,y=(1+bh).cumprod(),name='BH'))
fig.add_trace(go.Scatter(x=bh_eq.index,y=(1+bh_eq).cumprod(),name='BH EQRisk'))
fig.update_layout(title=f'TSM vs BH (Sharpe {sh_tsm:.2f} vs {sh_bh_eq:.2f})',yaxis_title='Equity')
fig.show()
print({'sharpe_tsm':sh_tsm,'sharpe_bh':sh_bh,'sharpe_bh_eq':sh_bh_eq})


{'sharpe_tsm': 0.9958942002480788, 'sharpe_bh': 0.9468079874159143, 'sharpe_bh_eq': 1.0225200598283}


## Detailed Statistics

### Performance Table

This table summarizes key metrics for TSM, BH, and BH EQRisk:

- **sharpe**: annualized Sharpe ratio
- **annual_return**: geometric annualized return (CAGR)
- **annual_vol**: annualized volatility (std × √periods/year)
- **max_drawdown**: maximum drawdown (peak-to-trough loss)

**Interpretation**:
- If `Sharpe(TSM) > Sharpe(BH_EQRisk)`: strategy beats Buy & Hold at equal risk ✅
- If `max_drawdown(TSM) < max_drawdown(BH_EQRisk)`: better risk management ✅
- If `annual_return(TSM) > annual_return(BH_EQRisk)`: better absolute return at same vol ✅

### Statistical Significance

For robust comparison, we should also compute:

**Information Ratio** (excess return per unit of tracking error):

$$\text{IR} = \frac{\mu_{\text{TSM}} - \mu_{\text{BH}}}{\text{std}(r_{\text{TSM}} - r_{\text{BH}})}$$

**Calmar Ratio** (return/max drawdown):

$$\text{Calmar} = \frac{\mu_p}{|\text{MDD}|}$$


In [37]:
def compute_stats(returns:pd.Series,freq:str='W')->Dict:
    s=returns.dropna()
    if s.empty:return {}
    ppy=52 if freq.upper().startswith('W') else 12
    ann_ret=((1.0+s).prod()**(ppy/len(s)))-1.0
    ann_vol=s.std()*np.sqrt(ppy)
    sharpe=ann_ret/ann_vol if ann_vol>0 else float('nan')
    cum=(1.0+s).cumprod()
    dd=(cum/cum.cummax())-1.0
    max_dd=dd.min()
    return {'sharpe':sharpe,'annual_return':ann_ret,'annual_vol':ann_vol,'max_drawdown':max_dd}

stats_tsm=compute_stats(oos_returns,rb)
stats_bh=compute_stats(bh,rb)
stats_bh_eq=compute_stats(bh_eq,rb)
pd.DataFrame({'TSM':stats_tsm,'BH':stats_bh,'BH_EQRisk':stats_bh_eq}).T


,sharpe,annual_return,annual_vol,max_drawdown
TSM,0.995894,0.319491,0.320808,-0.228954
BH,0.946808,0.550585,0.581517,-0.751519
BH_EQRisk,1.022520,0.328033,0.320808,-0.512669


## Strategy Optimization

### Current Issue

TSM Sharpe (0.99) < BH EQRisk Sharpe (1.02) → we need to improve risk-adjusted returns.

### Optimization Levers

1. **Reduce transaction costs**: lower turnover via longer rebalancing or higher smoothing
2. **More aggressive vol-targeting**: target 40-50% vol instead of 35%
3. **Remove trend filter**: it may be keeping us flat during profitable periods
4. **Shorter lookback**: crypto momentum decays faster than traditional assets (try 60-120 days)
5. **Pure momentum**: remove carry (w_mom=1.0, w_carry=0.0) if funding is noisy

Let's test an aggressive parameter grid:


# BTC Time Series Momentum Strategy

## Academic Foundation

Based on Moskowitz, Ooi & Pedersen (2012) “Time Series Momentum”, this strategy exploits the persistence of price trends (momentum) in financial markets.

Core principle: Assets that have performed well recently (1–12 month lookback) tend to continue their trajectory in the short/medium term.

### Mathematical Framework (MOP 2012)

The canonical TSM signal for asset \( i \) at time \( t \):
$$
h_{i,t} = \operatorname{sign}\!\big(r_{i,t-12,t}\big)\times \frac{\sigma_{\text{target}}}{\sigma_{i,t}}
$$

Where:
- \( r_{i,t-12,t} = \frac{P_{i,t}}{P_{i,t-12}} - 1 \): 12-month return (momentum signal)
- \( \sigma_{i,t} \): realized volatility (EWMA or rolling)
- \( \sigma_{\text{target}} \): target volatility (e.g., 40% annualized)
- \( h_{i,t} \): position size (leverage-adjusted)

Return attribution:
$$
r_{p,t} = \sum_i h_{i,t-1}\, r_{i,t} - \mathrm{TC}_{i,t}
$$

Where \( \mathrm{TC}_{i,t} = c \times \lvert h_{i,t} - h_{i,t-1} \rvert \) are transaction costs.

## Our Implementation

We extend the canonical TSM with crypto-specific enhancements:
1. Momentum Signal: \( r_{t-\text{lookback},t} \) with lookback ∈ [180, 252, 365] days
2. Trend Filter: SMA filter to avoid bear markets (long only if \( P_t > \mathrm{SMA}_{\text{trend}} \))
3. Carry (Funding): monetize perpetual funding rates (crypto-specific alpha)
4. Vol-Targeting: dynamic leverage to target 35% annualized vol
5. Position Smoothing: EWMA halflife 2–4 to reduce turnover
6. Transaction Costs: realistic 2 bps per trade

Objective: beat Buy & Hold at equal risk (same annualized volatility).

## TSM Strategy: compute_tsm_signals

### Strategy Logic with Mathematical Formulation

**Step 1: Momentum Signal**

\[
\text{mom}_t = \frac{P_t}{P_{t-\text{lookback}}} - 1
\]

**Step 2: Carry Signal (Crypto-Specific)**

Perpetual futures funding rates provide a carry component:

\[
\text{carry}_t = \sum_{s=t-w}^{t} f_s
\]

Where \(f_s\) is the funding rate at time \(s\) (typically paid every 8h on Binance), and \(w\) is the carry window (7-30 days).

**Funding Rate Mechanics**:
- **Positive funding** (\(f > 0\)): longs pay shorts → bullish sentiment → we capture this premium when long
- **Negative funding** (\(f < 0\)): shorts pay longs → bearish sentiment → we earn funding when long
- **Our strategy**: aggregate funding over `carry_window` days and combine with momentum

**Step 3: Composite Score**

\[
\text{score}_t = w_{\text{mom}} \times z(\text{mom}_t) + w_{\text{carry}} \times z(\text{carry}_t)
\]

Where \(z(x) = \frac{x - \mu_x}{\sigma_x}\) is the z-score normalization.

**Step 4: Directional Signal**

\[
\text{signal}_t = \begin{cases}
1 & \text{if } \text{score}_t > \theta \text{ and } P_t > \text{SMA}_{\text{trend}}(P) \\
0 & \text{otherwise}
\end{cases}
\]

**Step 5: Volatility Targeting**

\[
\sigma_{t} = \sqrt{\text{EWMA}_{\lambda}(r_t^2)} \times \sqrt{252}
\]

\[
\text{leverage}_t = \min\left(\frac{\sigma_{\text{target}}}{\sigma_t}, \text{lev}_{\text{max}}\right)
\]

**Step 6: Position with Smoothing**

\[
\text{pos}_t^{\text{raw}} = \text{signal}_{t-1} \times \text{leverage}_{t-1}
\]

\[
\text{pos}_t = \text{EWMA}_{\text{halflife}}(\text{pos}_t^{\text{raw}})
\]

**Step 7: Net Return**

\[
r_{p,t} = \text{pos}_{t-1} \times r_{t} - c \times |\text{pos}_t - \text{pos}_{t-1}| + \text{pos}_{t-1} \times f_t
\]

Where:
- \(r_t\): asset return
- \(c\): transaction cost (2bps)
- \(f_t\): funding rate received/paid

### Key Parameters

- `lookback_days`: momentum window (180-365)
- `trend_sma_days`: SMA for trend filter (150-252)
- `carry_window`: rolling sum window for funding (7-30 days)
- `w_mom`, `w_carry`: momentum vs carry weights (0.7/0.3)
- `score_threshold`: entry threshold (0.0 or 0.25)
- `target_vol`: target annualized volatility (35%)
- `max_leverage`: maximum leverage (3x)
- `position_ewm_halflife`: position smoothing (2-4)
- `transaction_cost_bps`: transaction costs (2bps)


In [38]:
def _z(x:pd.Series)->pd.Series:
    s=x.dropna();v=s.std()
    if v==0 or np.isnan(v):return x*0.0
    return (x-s.mean())/v

def compute_tsm_signals(prices:pd.Series,funding:Optional[pd.Series]=None,rebalance_freq:str='W',lookback_days:int=252,trend_sma_days:int=200,carry_window:int=14,w_mom:float=0.7,w_carry:float=0.3,score_threshold:float=0.0,vol_lambda:float=0.97,target_vol:float=0.35,vol_floor:float=0.15,max_leverage:float=3.0,transaction_cost_bps:float=2.0,position_ewm_halflife:Optional[float]=2.0)->pd.DataFrame:
    prices=prices.dropna().sort_index()
    r=prices.pct_change().dropna()
    ewma=np.sqrt(r.pow(2).ewm(alpha=1-vol_lambda,adjust=False).mean())
    sig_ann=(ewma*np.sqrt(252.0)).clip(lower=vol_floor)
    idx=prices.resample(rebalance_freq).last().index
    df=pd.DataFrame(index=idx)
    df['close']=prices.reindex(idx,method='ffill')
    df['asset_return']=df['close'].pct_change().fillna(0.0)
    mom=prices.pct_change(lookback_days).reindex(idx,method='ffill')
    trend=prices.rolling(trend_sma_days).mean().reindex(idx,method='ffill')
    if funding is not None and not funding.empty:
        carry=funding.rolling(f'{carry_window}D').sum().reindex(idx,method='ffill').fillna(0.0)
    else:
        carry=pd.Series(0.0,index=idx)
    score=w_mom*_z(mom)+w_carry*_z(carry)
    signal=((score>score_threshold)&(df['close']>trend)).astype(float).shift(1).fillna(0.0)
    sigma=sig_ann.resample(rebalance_freq).last().reindex(idx).shift(1).ffill().clip(lower=vol_floor)
    lev=(target_vol/sigma).replace([np.inf,-np.inf],np.nan)
    pos=(signal*lev).clip(-max_leverage,max_leverage).fillna(0.0)
    if position_ewm_halflife:pos=pos.ewm(halflife=position_ewm_halflife,adjust=False).mean()
    df['position']=pos
    df['turnover']=pos.diff().abs().fillna(pos.abs());cost=transaction_cost_bps/10000.0
    df['transaction_cost']=df['turnover']*cost
    fund=pd.Series(0.0,index=idx) if funding is None else funding.resample(rebalance_freq).sum().reindex(idx).fillna(0.0)
    df['funding_pnl']=fund*pos
    df['strategy_return_net']=pos*df['asset_return']-df['transaction_cost']+df['funding_pnl']
    df['strategy_equity']=(1.0+df['strategy_return_net']).cumprod()
    return df

@dataclass
class ParamSet:
    lookback_days:int
    trend_sma_days:int
    carry_window:int
    w_mom:float
    w_carry:float
    score_threshold:float
    rebalance_freq:str='W'
    vol_lambda:float=0.97
    target_vol:float=0.35
    vol_floor:float=0.15
    max_leverage:float=3.0
    transaction_cost_bps:float=2.0
    position_ewm_halflife:Optional[float]=2.0
    def to_kwargs(self)->Dict:
        return {'lookback_days':self.lookback_days,'trend_sma_days':self.trend_sma_days,'carry_window':self.carry_window,'w_mom':self.w_mom,'w_carry':self.w_carry,'score_threshold':self.score_threshold,'rebalance_freq':self.rebalance_freq,'vol_lambda':self.vol_lambda,'target_vol':self.target_vol,'vol_floor':self.vol_floor,'max_leverage':self.max_leverage,'transaction_cost_bps':self.transaction_cost_bps,'position_ewm_halflife':self.position_ewm_halflife}

def generate_param_grid()->List[ParamSet]:
    g=[]
    for lb in [180,252,365]:
        for tr in [150,200,252]:
            for cw in [7,14,30]:
                for thr in [0.0,0.25]:
                    g.append(ParamSet(lb,tr,cw,0.7,0.3,thr))
    return g

def compute_sharpe(returns:pd.Series,rebalance_freq:str)->float:
    s=returns.dropna()
    if s.empty:return float('nan')
    ppy=52 if rebalance_freq.upper().startswith('W') else 12
    ann=((1.0+s).prod()**(ppy/len(s)))-1.0
    vol=s.std()*np.sqrt(ppy)
    return float(ann/vol) if vol>0 else float('nan')
